# DSCI 525: Web and Cloud Computing

## Milestone 1: Tackling Big Data on Computer

### Group 13
Authors: Ivy Zhang, Mike Lynch, Selma Duric, William Xu

## Table of contents

- [Download the data](#1)
- [Combining data CSVs](#2)
- [Load the combined CSV to memory and perform a simple EDA](#3)
- [Perform a simple EDA in R](#4)
- [Reflection](#5)

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
import pyarrow.feather as feather
from memory_profiler import memory_usage
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import dask.dataframe as dd

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## 1. Download the data <a name="1"></a>

1. Download the data from figshare to local computer using the figshare API.
2. Extract the zip file programmatically.

In [3]:
# Attribution: DSCI 525 lecture notebook
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]    

In [5]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.99 s, sys: 5.06 s, total: 10.1 s
Wall time: 1min 44s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 19 s, sys: 5.1 s, total: 24.1 s
Wall time: 32.7 s


## 2. Combining data CSVs <a name="2"></a>

1. Use one of the following options to combine data CSVs into a single CSV (Pandas, Dask). **We used the option of Pandas**.
2. When combining the csv files, we added extra column called "model" that identifies the model (we get this column populated from the file name eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON)
3. Compare run times and memory usages of these options on different machines within the team, and summarize observations.

In [7]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
# use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 171.71 MiB, increment: 0.05 MiB
CPU times: user 7min 14s, sys: 30.9 s, total: 7min 45s
Wall time: 8min 11s


In [8]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv")

CPU times: user 1min 10s, sys: 26 s, total: 1min 36s
Wall time: 1min 46s


In [9]:
print(df.shape)

(62513863, 7)


In [10]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,figsharerainfall/MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,figsharerainfall/MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,figsharerainfall/MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,figsharerainfall/MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,figsharerainfall/MPI-ESM-1-2-HAM


**Summary of run times and memory usages:**

***William***
- Combining files: 
    - peak memory: 95.41 MiB, increment: 0.26 MiB
    - CPU times: user 7min 28s, sys: 31 s, total: 7min 59s
    - Wall time: 9min 17s
- Reading the combined file:
    - Wall time: 1min 51s

***Mike***
- Combining files: 
    - peak memory: 168.59 MiB, increment: 0.12 MiB
    - CPU times: user 3min 29s, sys: 5.09 s, total: 3min 34s
    - Wall time: 3min 34s
- Reading the combined file:
    - Wall time: 37.1 s

***Selma***
- Combining files: 
    - peak memory: 150.54 MiB, increment: 0.23 MiB
    - CPU times: user 6min 46s, sys: 23.1 s, total: 7min 9s
    - Wall time: 7min 29s
- Reading the combined file:
    - Wall time: 1min 19s
    
***Ivy***
- Combining files: 
    - peak memory: 156.23 MiB, increment: 0.00 MiB
    - CPU times: user 5min 14s, sys: 18.2 s, total: 5min 32s
    - Wall time: 5min 45s
- Reading the combined file:
    - Wall time: 1min 30s

## 3. Load the combined CSV to memory and perform a simple EDA <a name="3"></a>

### Establish a baseline for memory usage

In [11]:
# First load in the dataset using default settings for dtypes
df_eda = pd.read_csv("figsharerainfall/combined_data.csv", parse_dates=True, index_col='time')
df_eda.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,figsharerainfall/MPI-ESM-1-2-HAM


In [12]:
# As we can see below, dtypes are float64 and object
df_eda.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [13]:
# Measure the memory usage when representing numbers using float64 dtype
print(f"Memory usage with float64: {df_eda.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3500.78 MB


In [14]:
%%time
%memit

# Now perform a simple EDA with pandas describe function
df_eda.describe()

peak memory: 698.22 MiB, increment: 0.35 MiB
CPU times: user 16.2 s, sys: 13.8 s, total: 30 s
Wall time: 36.5 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.929456e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901827e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588275e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


Baseline memory and time data:
- Memory usage with float64: 3500.78 MB
- peak memory: 698.22 MiB, increment: 0.35 MiB
- CPU times: user 16.2 s, sys: 13.8 s, total: 30 s
- Wall time: 36.5 s

### Effects of changing dtypes on memory usage

In [15]:
# Now load in the dataset using float32 dtype to represent numbers
colum_dtypes = {'lat_min': np.float32, 'lat_max': np.float32, 'lon_min': np.float32, 'lon_max': np.float32, 'rain (mm/day)': np.float32, 'model': str}
df_eda = pd.read_csv("figsharerainfall/combined_data.csv",parse_dates=True, index_col='time', dtype=colum_dtypes)
df_eda.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439865,-33.574619,141.5625,143.4375,4.244226e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439865,-33.574619,141.5625,143.4375,4.217326e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439865,-33.574619,141.5625,143.4375,4.498125e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439865,-33.574619,141.5625,143.4375,4.251282e-13,figsharerainfall/MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439865,-33.574619,141.5625,143.4375,4.270161e-13,figsharerainfall/MPI-ESM-1-2-HAM


In [16]:
# As we can see below, dtypes are float32 and object
df_eda.dtypes

lat_min          float32
lat_max          float32
lon_min          float32
lon_max          float32
rain (mm/day)    float32
model             object
dtype: object

In [17]:
print(f"Memory usage with float32: {df_eda.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float32: 2250.50 MB


In [18]:
%%time
%memit

# Now perform a simple EDA with pandas describe function
df_eda.describe()

peak memory: 800.12 MiB, increment: 0.39 MiB
CPU times: user 10.5 s, sys: 5.21 s, total: 15.7 s
Wall time: 19.8 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.929456e+07
mean,-3.310497e+01,-3.197765e+01,1.469058e+02,1.482150e+02,1.901828e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588274e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


Time and memory data when using different dtypes:
- Memory usage with float32: 2250.50 MB
- peak memory: 800.12 MiB, increment: 0.39 MiB
- CPU times: user 10.5 s, sys: 5.21 s, total: 15.7 s
- Wall time: 19.8 s

### Effects of loading a smaller subset of columns on memory usage

In [19]:
# Now load only a subset of columns from the dataset
df_eda = pd.read_csv("figsharerainfall/combined_data.csv",parse_dates=True, index_col='time', usecols=['time', 'lat_min', 'rain (mm/day)'])
df_eda.head()

,lat_min,rain (mm/day)
time,,
1889-01-01 12:00:00,-35.439867,4.244226e-13
1889-01-02 12:00:00,-35.439867,4.217326e-13
1889-01-03 12:00:00,-35.439867,4.498125e-13
1889-01-04 12:00:00,-35.439867,4.251282e-13
1889-01-05 12:00:00,-35.439867,4.270161e-13


In [20]:
# As we can see below, dtypes are float64 by default
df_eda.dtypes

lat_min          float64
rain (mm/day)    float64
dtype: object

In [21]:
print(f"Memory usage with reduced number of columns: {df_eda.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with reduced number of columns: 1500.33 MB


In [22]:
%%time
%memit

# Now perform a simple EDA with pandas describe function
df_eda.describe()

peak memory: 848.64 MiB, increment: 0.35 MiB
CPU times: user 6.41 s, sys: 4.26 s, total: 10.7 s
Wall time: 15 s


,lat_min,rain (mm/day)
count,5.924854e+07,5.929456e+07
mean,-3.310482e+01,1.901827e+00
std,1.963549e+00,5.588275e+00
min,-3.646739e+01,-3.807373e-12
25%,-3.486911e+01,3.876672e-06
50%,-3.300000e+01,6.161705e-02
75%,-3.140170e+01,1.021314e+00
max,-2.990000e+01,4.329395e+02


Time and memory data when using column subset:
- Memory usage with reduced number of columns: 1500.33 MB
- peak memory: 848.64 MiB, increment: 0.35 MiB
- CPU times: user 6.41 s, sys: 4.26 s, total: 10.7 s
- Wall time: 15 s

### Summary

#### Using float32 vs. baseline float64 dtype to perform a simple EDA:
- The memory usage decreased from 3500.78 MB to 2250.50 MB when representing numbers using float32 instead of float64
- When using the pandas describe function to perform a simple EDA, we found that the peak memory increased when using float32 dtype for the numerical columns. 
- The wall time taken to perform the EDA also decreased substantially to 19.8s from the baseline of 36.5s. 

#### Using a reduced number of columns compared to the baseline to perform a simple EDA:
- The memory usage decreased from 3500.78 MB to 1500.33 MB when using a subset of columns from the dataset
- When using the pandas describe function to perform a simple EDA, we found that the peak memory increased when using fewer columns. 
- The wall time taken to perform the EDA also decreased substantially to 15s from the baseline of 36.5s. 

## 4. Perform a simple EDA in R <a name="4"></a>

We will transform our dataframe into different formats before loading into R.
#### I. Default memory format + feather file format

In [23]:
%%time
feather.write_feather(df, "figsharerainfall/combined_data.feather")

CPU times: user 15.1 s, sys: 22.3 s, total: 37.4 s
Wall time: 31.6 s


#### II. dask + parquet file format

In [24]:
ddf = dd.read_csv("figsharerainfall/combined_data.csv")

In [25]:
%%time
dd.to_parquet(ddf, 'figsharerainfall/combined_data.parquet')

CPU times: user 2min 22s, sys: 38.4 s, total: 3min 1s
Wall time: 1min 25s


#### III. Arrow memory format + parquet file format

In [26]:
%%time
%%memit
dataset = ds.dataset("figsharerainfall/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 1783.19 MiB, increment: 1130.65 MiB
CPU times: user 25.5 s, sys: 17.4 s, total: 42.9 s
Wall time: 39.2 s


In [27]:
%%time
pq.write_to_dataset(table, 'figsharerainfall/rainfall.parquet')

CPU times: user 17.3 s, sys: 9.28 s, total: 26.6 s
Wall time: 50.7 s


#### IV. Arrow memory format + feather file format

In [28]:
%%time
feather.write_feather(table, 'figsharerainfall/rainfall.feather')

CPU times: user 5.44 s, sys: 11.3 s, total: 16.7 s
Wall time: 17.4 s


In [29]:
%%sh
du -sh figsharerainfall/combined_data.csv
du -sh figsharerainfall/combined_data.parquet
du -sh figsharerainfall/rainfall.parquet
du -sh figsharerainfall/rainfall.feather

6.6G	figsharerainfall/combined_data.csv
2.3G	figsharerainfall/combined_data.parquet
540M	figsharerainfall/rainfall.parquet
1.0G	figsharerainfall/rainfall.feather


### Transfer different formats of data from Python to R

It is usually not efficient to directly transfer Python dataframe to R due to serialization and deserialization involved in the process. Also, we observe Arrow memory format performs better than the default memory default. Thus, our next step is to further compare the performance of transferring Arrow-feather file and Arrow-parquet file to R.

#### I. Read Arrow-parquet file to R

```python
%%time
%%R
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_parquet("figsharerainfall/rainfall.parquet/e5a0076fe71f4bdead893e20a935897b.parquet")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)
```

![](../img/output.png)

*Note that the code above has been commented out to ensure the workbook is reproducible. Please check Reflection for more details*

#### II. Read Arrow-feather file to R

In [30]:
%%time
%%R
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figsharerainfall/rainfall.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 28 x 2
   model                                   n
 * <chr>                               <int>
 1 figsharerainfall/ACCESS-CM2       1932840
 2 figsharerainfall/ACCESS-ESM1-5    1610700
 3 figsharerainfall/AWI-ESM-1-1-LR    966420
 4 figsharerainfall/BCC-CSM2-MR      3035340
 5 figsharerainfall/BCC-ESM1          551880
 6 figsharerainfall/CanESM5           551880
 7 figsharerainfall/CMCC-CM2-HR4     3541230
 8 figsharerainfall/CMCC-CM2-SR5     3541230
 9 figsharerainfall/CMCC-ESM2        3541230
10 figsharerainfall/EC-Earth3-Veg-LR 3037320
# … with 18 more rows
Time difference of 1.11685 mins
CPU times: user 18.4 s, sys: 32.2 s, total: 50.6 s
Wall time: 1min 8s


#### Summary of format selection
Based on the data storage and processing time comparison from above, our preferred format among all is **parquet using Arrow package**. The file with this format takes much less space to store it. Also, it takes less time to write to this format and read it in R.

## Reflection <a name="5"></a>

After some trial and error, all team members were individually able to successfully run the analysis from start to finish, however during the process we did experience some problems which included the following:
- William had issue with `%load_ext rpy2.ipython` despite the successful environment installation on his MacOS. After many hours debugging, ry2 finally worked after specifying the python version in the course yml file. The solution is to add `python=3.8.6` to the 525.yml file under `dependencies:` and reinstall the environment. 
- Even though the file sizes were only 5 GB, we actually required 10 GB of disk space since we needed to download and unzip the data.
- We got some confusing results by accidentally re-downloading the dataset without first deleting it since we were then combining twice as many files in the next step.
- We noticed that parquet file name was not consistently generated when we re-run the workbook. If we keep the curent file name and re-run all cells, `arrow::read_parquet` function would return an error message indicating that the file "e5a0076fe71f4bdead893e20a935897b.parquet" does not exist in the directory. For reproducibility reason, we decided to comment out the code but record the output for further comparison.